In [0]:
!pip install --upgrade pip
!pip install rouge
!pip install nltk
!pip install rake_nltk
!pip install autocorrect
!pip install networkx

     |████████████████████████████████| 2.1 MB 5.7 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.0.1
    Uninstalling pip-21.0.1:
      Successfully uninstalled pip-21.0.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 KB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 KB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.6/764.6 KB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 KB 15.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622380 sha256=741f56d511394f8551a335b939a54ed5d73a948165cf8396cf263efe524574b6
  Stored in directory: /root/.cache/pip/wheels/72/b8/3b/a90246d13090e85394a8a44b78c8abf577c0766f29d6543c75
Successfully built autocorrect
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9

In [0]:
import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Out[2]: True

In [0]:
import re
import copy
import math
import time
import pickle
import string
import numpy as np
import pandas as pd
import networkx as nx
from pyspark import *
from rouge import Rouge
from nltk import pos_tag
from pyspark.sql import *
from graphframes import *
from rake_nltk import Rake
from datetime import datetime
from pyspark.sql.types import *
from autocorrect import Speller
from nltk.corpus import stopwords
from matplotlib.pyplot import figure
from matplotlib import pyplot as plt
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')

Out[3]: <Figure size 640x480 with 0 Axes><Figure size 640x480 with 0 Axes>

In [0]:
# Document to process
no_documents = 20

In [0]:
class misc_functions:
    def __init__(self):
        pass
      
    def word_count_dict_generator(self, count_matrix, words):
        word_count_dict = {}
        word_count_list = count_matrix.sum(axis=0)
        for i in range(0, len(words)):
            word_count_dict[words[i]] = word_count_list[i]
        return word_count_dict
        
    def jaccard_simlarity(self,sen1_lemma,sen2_lemma):
        s1_set = set(sen1_lemma)
        s2_set = set(sen2_lemma)
        similarity = float(len(s1_set & s2_set))/float(len(s1_set | s2_set))
        return round(similarity,3)
    
    def table_to_sentence_list(self, table_name, low_doc_id = 0, upper_doc_id = no_documents):
    #    i/p: csv_name: name of csv file
    #             type: string
    #    o/p: sentence_lists: list of list of sentences document wise
    #             type: list
    #             ex: [ [s1,s2,s3,s4], [s1,s2], [s1,s2,s3,s4,s5]]
    #                     doc 1         doc 2         doc 3
    #         low_doc_id: lower limit to retrive document.
    #             type: int
    #         upper_doc_id: upper limit to retrive document.
    #             type: int
        sentence_lists = []
        wiki_how_all_sentences = sqlContext.sql("SELECT * FROM " + table_name +  " WHERE " + "doc_id >= " + str(low_doc_id) + " AND " + "doc_id < " + str(upper_doc_id))
        column_list = wiki_how_all_sentences.collect()
        doc_id_list = []
        count = 0
        for row in column_list:
#           print(str(row['doc_id']) + str(count))
          try:
            doc_id_list.append(int(row['doc_id']))
          except TypeError:
#             print(str(row['doc_id']) + " " + str(count)+ " "+ str(row['sen_id']))
            continue
          count += 1
#         df = pd.read_csv(dataset_path + csv_name)
        for i in range(low_doc_id, upper_doc_id):
            doc = wiki_how_all_sentences.filter(wiki_how_all_sentences.doc_id==i)
            doc_column_list = doc.collect()
            sentence_lists.append([str(row['sentence']) for row in doc_column_list])
        return sentence_lists
      
    def get_contractions(self):
        contractions = {"'cause": 'because', "ain't": 'am not', "aren't": 'are not', "can't": 'cannot', "can't've": 'cannot have', "could've": 'could have', "couldn't": 'could not', "couldn't've": 'could not have', "didn't": 'did not', "doesn't": 'does not', "don't": 'do not', "hadn't": 'had not', "hadn't've": 'had not have', "hasn't": 'has not', "haven't": 'have not', "he'd": 'he had', "he'd've": 'he would have', "he'll": 'he will', "he'll've": 'he will have', "he's": 'he is', "how'd": 'how did', "how'd'y": 'how do you', "how'll": 'how will', "how's": 'how is', "i'd": 'I had', "i'd've": 'I would have', "i'll": 'I will', "i'll've": 'I will have', "i'm": 'I am', "i've": 'I have', "isn't": 'is not', "it'd": 'it had', "it'd've": 'it would have', "it'll": 'it will', "it'll've": 'it will have', "it's": 'it is', "let's": 'let us', "ma'am": 'madam', "mayn't": 'may not', "might've": 'might have', "mightn't": 'might not', "mightn't've": 'might not have', "must've": 'must have', "mustn't": 'must not', "mustn't've": 'must not have', "needn't": 'need not', "needn't've": 'need not have', "o'clock": 'of the clock', "oughtn't": 'ought not', "oughtn't've": 'ought not have', "sha'n't": 'shall not', "shan't": 'shall not', "shan't've": 'shall not have', "she'd": 'she had', "she'd've": 'she would have', "she'll": 'she will', "she'll've": 'she will have', "she's": 'she is', "should've": 'should have', "shouldn't": 'should not', "shouldn't've": 'should not have', "so's": 'so is', "so've": 'so have', "that'd": 'that had', "that'd've": 'that would have', "that's": 'that is', "there'd": 'there had', "there'd've": 'there would have', "there's": 'there is', "they'd": 'they had',
"they'd've": 'they would have', "they'll": 'they will', "they'll've": 'they will have', "they're": 'they are', "they've": 'they have', "to've": 'to have', "wasn't": 'was not', "we'd": 'we had', "we'd've": 'we would have', "we'll": 'we will', "we'll've": 'we will have', "we're": 'we are', "we've": 'we have', "weren't": 'were not', "what'll": 'what will', "what'll've": 'what will have', "what're": 'what are', "what's": 'what is', "what've": 'what have', "when's": 'when is', "when've": 'when have', "where'd": 'where did', "where's": 'where is', "where've": 'where have', "who'll": 'who will', "who'll've": 'who will have', "who's": 'who is', "who've": 'who have', "why's": 'why is', "why've": 'why have', "will've": 'will have', "won't": 'will not', "won't've": 'will not have', "would've": 'would have', "wouldn't": 'would not', "wouldn't've": 'would not have', "y'all": 'you all', "y'all'd": 'you all would', "y'all'd've": 'you all would have', "y'all're": 'you all are', "y'all've": 'you all have',
"you'd": 'you had', "you'd've": 'you would have', "you'll": 'you will', "you'll've": 'you will have',"you're": 'you are',
"you've": 'you have'}
        return contractions
      
    def plot_graph(self, graph):
        edge_list= graph.edges
        Gplot=nx.Graph()
        for row in edge_list.select('src','dst').take(1000):
            Gplot.add_edge(row['src'],row['dst'])
        plt.subplot(121)
        nx.draw(Gplot, with_labels=True, font_weight='bold')
        return
      
    def checkpoint(self, gcs,last_finished):
        file =  open('gcs0', 'wb')
        pickle.dump(gcs, file)
        file =  open('lastcp0', 'wb')
        pickle.dump(last_finished, file)
        file.close()
        return
      
    def calculate_rank_cluster(self, vertex_id, graph_obj):
        rank = 0.0
        inter_graph = graph_obj.filterEdges("dst == " + str(vertex_id))
        edges_df = inter_graph.edges.collect()
        for row in edges_df:
          rank+=float(row['relationship'])
      
        inter_graph = graph_obj.filterEdges("src == " + str(vertex_id))
        edges_df = inter_graph.edges.collect()
        for row in edges_df:
          rank+=float(row['relationship'])
      
        return round(rank,4)  
      
    def rankwise_sort(self, sentence_list, sentence_ranks):
        sorted_list = []
        ranks = []
        
        for i in sentence_list:
            ranks.append(sentence_ranks[i])
            
        for i in range(len(ranks)):
            max_index = ranks.index(max(ranks))
            ranks[max_index] = float('-inf')
            sorted_list.append(sentence_list[max_index])
            
        return sorted_list
      
    def print_rouge_statistics(self, rouge_statistics):
        print("------------------------------------------------------------------------------------")
        print("|       |  ROUGE-1  |  Document  |  ROUGE-2  |  Document  |  ROUGE-L  |  Document  |")
        print("------------------------------------------------------------------------------------")
        print("|  AVG  |   "+str(rouge_statistics['rouge-1']).ljust(6, '0')+"  |     NA     |   "+str(rouge_statistics['rouge-2']).ljust(6, '0')+"  |     NA     |  "+str(rouge_statistics['rouge-l']).ljust(6, '0')+"   |     NA     |")
        print("------------------------------------------------------------------------------------")
        print("|  MIN  |   "+str(rouge_statistics['min_rouge_1']).ljust(6, '0')+"  |     "+str(rouge_statistics['min_rouge_1_doc_id']).rjust(2, '0')+"     |   "+str(rouge_statistics['min_rouge_2']).ljust(6, '0').ljust(6, '0')+"  |     "+str(rouge_statistics['min_rouge_2_doc_id']).rjust(2, '0')+"     |  "+str(rouge_statistics['min_rouge_l']).ljust(6, '0')+"   |     "+str(rouge_statistics['min_rouge_l_doc_id']).rjust(2, '0')+"     |")
        print("------------------------------------------------------------------------------------")
        print("|  MAX  |   "+str(rouge_statistics['max_rouge_1']).ljust(6, '0')+"  |     "+str(rouge_statistics['max_rouge_1_doc_id']).rjust(2, '0')+"     |   "+str(rouge_statistics['max_rouge_2']).ljust(6, '0')+"  |     "+str(rouge_statistics['max_rouge_2_doc_id']).rjust(2, '0')+"     |  "+str(rouge_statistics['max_rouge_l']).ljust(6, '0')+"   |     "+str(rouge_statistics['max_rouge_l_doc_id']).rjust(2, '0')+"     |")
        print("------------------------------------------------------------------------------------")
        return

# Data Preprocessing

### Preprocessing

In [0]:
class data_preprocessing:
    def __init__(self):
        pass
    def early_preprocessing(self, doc_sentence_list):
    #    i/p: doc_sentence_list: list of all senteneces in single document
    #             type: list
    #    o/p: early_porced_doc_sentence_list: list of all processed sentences of a single document
    #             type: list
    #    This methods performs lower casing, contractions removal, puctuation removal, 
    #    white noise removal
        mf = misc_functions()
        contractions = mf.get_contractions()
        
        early_porced_doc_sentence_list = []
        
        for sentence in doc_sentence_list:
            #     Lower Casing
            sentence = sentence.lower()   
            #     Convert he'll -> he will
            for word in sentence.split():
                if word in contractions:
                    sentence = sentence.replace(word, contractions[word])
                    
            if re.search("\\w*'s", sentence ):
                temp = sentence
                sentence = ""
                for i in temp.split():
                    if re.search("\\w*'s", i ):
                        i = i.replace("'s","")
                    sentence += " " + i 
                    
            #    REMOVE PUNCTUATION
            translator = str.maketrans('', '', string.punctuation)
            sentence = sentence.translate(translator)
             
            #     REMOVE WHITE NOISE
            sentence = " ".join(sentence.split())
            early_porced_doc_sentence_list.append(sentence)
        return early_porced_doc_sentence_list
        
    def word_tokenization(self, early_porced_doc_sentence_list):
    #    i/p: early_porced_doc_sentence_list: list of all processed senteteces in single document using 'early_preprocessing_wikihow' method
    #             type: list
    #    o/p: sentence_tokens_list: list of list containing tokens of sentences
    #             type: list
    #    example: i/p: ['this example','this test']
    #             o/p: [['this','example'],['this','test']]
        no_sentences = len(early_porced_doc_sentence_list)
        sentence_tokens_list = []
        for sentence in early_porced_doc_sentence_list:
            sentence_tokens_list.append(word_tokenize(sentence))
        return sentence_tokens_list    
    
    def lemmatization(self, sentence_tokens_list):
    #    i/p: sentence_tokens_list: list of list containing tokens of sentences
    #             type: list
    #    o/p: sentence_lemma_list: list of list containing lemmas of individual sentence
    #             type: list
    #    example: i/p: [['i', 'am', 'playimg'], ['they', 'have', 'studied']]
    #             o/p: [['i', 'be', 'play'], ['they', 'have', 'study']]
        lemmatizer = WordNetLemmatizer()
        sentence_lemma_list = []
        for token_list in sentence_tokens_list:
            lemmas = [lemmatizer.lemmatize(word, pos ='v') for word in token_list]
            lemmas = [e for e in lemmas if e.isalnum()]
            if lemmas != []:
                sentence_lemma_list.append(lemmas)
        return sentence_lemma_list
    
    def stop_word_removal(self, sentence_lemma_list):
    #    i/p: sentence_lemma_list: list of list containing lemmas of individual sentence
    #             type: list
    #    o/p: sentence_lemma_list: list of list containing lemmas of individual sentence
    #         but without stop words.
    #             type: list
    #    example: i/p: [['i','am','playing'],['they','have','study','hard']]
    #             o/p: [['play'], ['hard','study']]
        stop_words = set(stopwords.words('english'))
        for index in range(0, len(sentence_lemma_list)):
            if sentence_lemma_list[index] == []:
              continue
            temp = list(set(sentence_lemma_list[index])-stop_words)
            if temp != []:
                sentence_lemma_list[index] = temp
        return sentence_lemma_list
    
    def pos_tagging(self, sentence_lemma_list):
    #    i/p: sentence_lemma_list: list of list containing lemmas of individual sentence
    #         after stopword removal
    #             type: list
    #    o/p: nouns: list of nouns in a document
    #             type: list
    #         words: list of unique words in a document
    #             type: list
    #    example: i/p: [['i', 'be', 'play','extra','playground'], 
    #                   ['they', 'have', 'study','hard','extra','exam','college']]
    #             o/p: Nouns: ['play', 'college', 'playground'], 
    #                  Words: ['play', 'college', 'extra', 'be', 'study', 'exam', 'i', 'have', 'they', 'hard', 'playground']
        nouns = []
        words = []
        for lemma_list in sentence_lemma_list:
            words = list(set(words).union(set(lemma_list)))
        for i in pos_tag(words):
            if (i[1] == 'NN' or i[1] == 'NNP') and i[0] not in nouns:
                nouns.append(i[0])
        return nouns, words

### Feature Extraction

In [0]:
class feature_extraction:
    mf = None
    def __init__(self):
        self.mf = misc_functions()
        pass
      
    def word_freq_counter(self, sentence_lemma_list, words):
    #    i/p: sentence_lemma_list: list of list containing lemmas of individual sentence
    #         after stopword removal
    #             type: list
    #         words: list of unique words in a document
    #             type: list
    #    o/p: count_matrix: count of each word per sentence
    #             row: sentences    column: words
    #             type: numpy array
    #    example: i/p: [['i', 'i' , 'be', 'play','extra','playground'], 
    #                  ['they', 'have', 'study','have','college']], 
    #                  ['play', 'college', 'extra', 'be', 'study', 'i', 'have', 'they', 'playground']
    #             o/p: [[1. 0. 1. 1. 0. 2. 0. 0. 1.]
    #                  [0. 1. 0. 0. 1. 0. 2. 1. 0.]]
        no_sentences = len(sentence_lemma_list)
        no_words = len(words)
        count_matrix = np.empty([no_sentences, no_words])
        for i in range(0,no_sentences):
            lemmas = sentence_lemma_list[i]
            for j in range(0,no_words):
                lemma = words[j]
                count_matrix[i][j] = lemmas.count(lemma)
        return count_matrix
    
    def tf_idf_score(self,count_matrix):
    #    i/p: count_matrix: count of each word per sentence
    #             row: sentences    column: words
    #             type: numpy array
    #    o/p: tf_idf_scores: tf-idf score of each word
    #             type: list
    #    Total number of words in the document
        total_words_in_document = count_matrix.sum()
    #    Number of times word appears in a sentence
        word_count = count_matrix.sum(axis=0)
    #    TF(word) = (Number of times word appears in a sentence) / (Total number of words in the document).
        tf_scores = [ i/float(total_words_in_document) for i in word_count]
    #    Total number of sentences
        total_no_sentences = count_matrix.shape[0]
    #    Number of sentences with word in it
        no_sentences = np.count_nonzero(count_matrix,axis=0)
    #    IDF(word) = log_e(Total number of sentences / Number of sentences with word in it)
        idf_scores_wo_log = [float(total_no_sentences)/i for i in no_sentences]
        idf_scores = [math.log(i) for i in idf_scores_wo_log]
    #    tf-idf = tf*idf
        tf_idf_scores = [i*j for i,j in zip(tf_scores,idf_scores)]
        return tf_idf_scores
    
    def keyword_extraction(self, method, input_parameter, words = []):
    #    i/p: method: method name to extract keyword TF-TDF or RAKE
    #             type: stirng
    #         input_parameter: if TFIDF -> list of tf-idf score of each words
    #                          if RAKE -> list of sentence lemmas in a single document   
    #         words: list of unique words in a document (only for tf-idf)
    #             type: list
    #    o/p: keywords: keywords/phrases list for given document.
    #             type: list
        if method == "tfidf":
            tf_keywords = []
            tf_idf_scores = copy.deepcopy(input_parameter)
            for i in range(0, len(tf_idf_scores)):
                # Getting max score from list                
                max_score = max(tf_idf_scores)
                # Getting index of max score
                max_score_index = tf_idf_scores.index(max_score)
                # Getting word with tthe max score
                word = words[max_score_index]
                # Append word in final list
                tf_keywords.append(word)
                # Assigning samallest value to already appended word
                # To prevent the selection of the same word in the next iteration
                tf_idf_scores[max_score_index] = float('-inf')
            return tf_keywords
        elif method == "rake":
            kw_sentence_lemma_list = copy.deepcopy(input_parameter)
            text = ""
            for lemma_list in kw_sentence_lemma_list:
                for lemma in lemma_list:
                    text += " " + lemma
                text += ". "
            r = Rake()
            # Getting score
            r.extract_keywords_from_text(text)
            # Sorting words based on the scores
            rake_keyphrases = r.get_ranked_phrases()
            return rake_keyphrases
    
    def sentenece_similarity_calculator(self,sentence_lemma_list):
    #    i/p: sentence_lemma_list: method name to extract keyword TF-TDF or RAKE
    #             type: list
    #    o/p: similarity_matrix: numpy array of similarity score between two sentences.
    #             type: numpy array
        no_sentences = len(sentence_lemma_list)
        similarity_matrix = np.empty([no_sentences, no_sentences])
        for i in range(0, no_sentences):
            for j in range(i, no_sentences):
                if i == j:
                    similarity_matrix[i][j] = 1
                    continue
                similarity_score = self.mf.jaccard_simlarity(sentence_lemma_list[i],sentence_lemma_list[j])
                similarity_matrix[i][j] = similarity_score
                similarity_matrix[j][i] = similarity_score
        return similarity_matrix
    
    def word_weight_calculation(self, nouns, keywords, count_matrix, words):
    #    i/p: nouns: list of nouns in a document
    #             type: list
    #         keywords: list of keywords/phrases in in a document
    #             type: list
    #         count_matrix: count of each word per sentence
    #             row: sentences    column: words
    #             type: numpy array 
    #         keywords: list of words in a document
    #             type: list
    #    o/p: keywords: keywords list for given document.
    #             type: list
        word_weight_dict = {}
        noun_count = []
        keyword_count = []
        total_noun_count = 0
        total_keyword_count = 0
        word_count_dict = self.mf.word_count_dict_generator(count_matrix, words)
        temp_index = 0
        #    Getting frequency of each noun in a document     
        for noun in nouns:
            total_noun_count += word_count_dict[noun]
        #    Getting frequency of each keyword in a document     
        for keyphrase in keywords:
            for keyword in keyphrase.split(" "):
                if keyword.isalnum():
                    total_keyword_count += word_count_dict[keyword]
       #    Calculation of each noun in a document          
        for noun in nouns:
            word_weight_dict[noun] = word_count_dict[noun]/float(total_noun_count)
            
        for keyphrase in keywords:
            for keyword in keyphrase.split(" "):
                if keyword.isalnum():
                    if keyword in word_weight_dict and keyword in nouns:
                        word_weight_dict[keyword] += word_count_dict[keyword]/float(total_keyword_count)
                        continue
                    if keyword in word_weight_dict and keyword not in nouns:
                        continue
                    word_weight_dict[keyword] = word_count_dict[keyword]/float(total_keyword_count)
                
        return word_weight_dict

# Processing

In [0]:
class rank_calculation:
    def __init__(self):
        return
    
    def graph_generation(self, similarity_matrix):
    #  1. ADDED REMOVAL OF EDGE WITH 0 WEIGHT  
    #  2. ONLY ONE SIDED EDGE IS USED      
    #  3. THRESOLD IS SET TO 75% Q3
    
    
    #    i/p: similarity_matrix: numpy array of similarity score between two sentences.
    #             type: numpy array
    #    o/p: sentence_graph: sentence based graph
    #             type: graphframe
    #             vertces: sentence id
    #             edges: jaccard similarity
    
    
        no_sentences = similarity_matrix.shape[0]
        
        # Checking for identity matrix
        row_ele_sum = np.sum(similarity_matrix, axis = 1)
        
        if no_sentences == np.sum(row_ele_sum):
          return None
        
        # Generating list [0, 1, 2, 3, 4, 5, ...., no_sentences-1]
        vertices_list = [i for i in range(0, no_sentences)]
#         print(vertices_list)
        # Generating Spark SQL DataFrame from list
        vertices = spark.createDataFrame(vertices_list, IntegerType())
        # Changing column name value->id. 'value' is default one.
        vertices = vertices.withColumnRenamed("value","id")
        # Defining structure of Edge Spark DataFrame -> (src: source (sentence_id), dst: destination (sentence_id), relationship: similarity score)
        edges_schema = StructType([StructField("src", IntegerType()),StructField("dst", IntegerType()),StructField("relationship", FloatType())])
        edges_list = []
        inter_edges_list = []
        weights = []
        # Generating edge_list to convert it into Spark SQL DataFrame:
        # Where each element (child list) in list (parent) represents single row of Edge Spark DataFrame 
        # Ex: [[0, 1, 0.2], [0, 2, 0.6], [1, 0, 0.2], [1, 2, 0.5], [2, 0, 0.6],[2, 1, 0.5]]
        for i in range(0, no_sentences):
          for j in range(i+1, no_sentences):
            if i==j or float(similarity_matrix[i][j]) == 0.0:
              continue
            inter_edges_list.append([i,j,float(similarity_matrix[i][j])])
            weights.append(float(similarity_matrix[i][j]))
        
        
        # Uncomment This Part TO Remove edges WRT thresold
        weights.sort()
        thresold = np.percentile(weights, 75, interpolation='midpoint')
        edges_list = [i for i in inter_edges_list if i[2] > thresold]
        
#         # Uncomment This Part TO not have thresold
#         edges_list = inter_edges_list
        
        # Generating Spark SQL DataFrame from edges_list  
        edges = spark.createDataFrame(edges_list,schema=edges_schema)
        # Generating Graph
        sentence_graph = GraphFrame(vertices, edges)
        
        return sentence_graph
        
      
    def eq_2(self, A, P, d):
    #    i/p: A: Similarity matrix between sentences
    #             type: numpy array
    #             row: sentences    column: senteces
    #         P: List of ranks of sentences
    #             type: list
    #         d: Damping Factor
    #             type: 0<d<1
    #    o/p: new_P: List of calculated ranks of sentences
    #             type: list
        new_P = copy.deepcopy(P)
        for i in range(0, len(P)):          #no_sentences == len(P)
            rank = 1 - d
            for j in range(0, len(P)):
                if j == i or A[i][j] == 0:
                    continue
        #        a_j_k = A.sum(axis = 0)[j]
                a_j_k = A[j].sum()
                rank += (d)*((float(A[i][j])/float(a_j_k))*new_P[j])
            new_P[i] = rank
        return new_P

    def eq_3(self, B, P, words, word_weight_dict):
    #    i/p: B: Normalized count matrix of words per sentences
    #             row: sentences    column: words
    #             type: numpy array
    #         P: List of ranks of sentences
    #             type: list
    #    o/p: Q: List of word weightage
    #             type: list
        Q = []
        for j in range(0,B.shape[1]):      #B.shape[1] == number of words in document
            num = 0
            dino = B.sum(axis = 0)[j]
            word = words[j]
            for i in range(0,len(P)):      #no_sentences == len(P)
                num += B[i][j]*P[i]
            sentence_based_weight = float(num)/float(dino)
            word_dict_weight = 0
#             if word in word_weight_dict:
#                 word_dict_weight = word_weight_dict[word]
            Q.append(sentence_based_weight + word_dict_weight)
        return Q
    
    def eq_4(self, B,Q):
    #    i/p: B: Normalized count matrix of words per sentences
    #             row: sentences    column: words
    #             type: numpy array
    #         Q: List of word weightage
    #             type: list
    #    o/p: P_star: List of calculated ranks of sentences based on word weight
    #             type: list
        P_star = []
        no_sentences = B.shape[0]
        for i in range(0,no_sentences):
            num = 0
            dino = B[i].sum()
            for j in range(0,len(Q)):      #number of words in document == len(Q)
                num += B[i][j]*Q[j]
            P_star.append(float(num)/float(dino))
        return P_star
    
    def eq_5(self, P,P_star,alpha = 0.6):
    #    i/p: P: List of ranks of sentences
    #             type: list
    #         P_star: List of calculated ranks of sentences based on word weight
    #             type: list
    #         alpha: Ratio between page rank value from eq_2 and eq_4
    #             type: 0<alpha<1
    #    o/p: P: List of calculated ranks of sentences or linear combination eq_2 and eq_4
    #             type: list
        for i in range(0,len(P)):
            P[i] = alpha*P[i] + (1-alpha)*P_star[i]
        return P
    
    def get_diff(self, P, P_tminus1):
    #    i/p: P: List of ranks of sentences from current iteration
    #             type: list
    #         P_tminus1: List of ranks of sentences from previous iteration
    #             type: list
    #    o/p: diff: Linear difference between P and P_tminus1
    #             type: float
        P_array = np.array(P)
        P_tminus1_array = np.array(P_tminus1)
        diff = (np.linalg.norm(P_array - P_tminus1_array))**2
        return diff
    
    def get_A(self, similarity_matrix):
    #    i/p: similarity_matrix: matrix of simalarity scores between sentences
    #             type: numpy array
    #    o/p: A: Copy of similarity matrix
    #             row: sentences    column: sentences
    #             type: numpy array
        A = copy.deepcopy(similarity_matrix)
        return A
    
    def get_B(self, C):
    #    i/p: C: matrix of count of words per sentence
    #             row: sentences    column: words
    #             type: numpy array
    #    o/p: B: Normalized count matrix of words per sentences
    #             type: numpy array
        B = np.empty([C.shape[0], C.shape[1]])
        for i in range(0, B.shape[0]):
            row_sum = C[i].sum()
            for j in range(0, B.shape[1]):
                B[i][j] = float(C[i][j])/float(row_sum)
        return B
    
    def get_C(self, count_matrix):
    #    i/p: count_matrix: matrix of count of words per sentence
    #             row: sentences    column: words
    #             type: numpy array
    #    o/p: C: Copy of count matrix
    #             type: numpy array
        C = copy.deepcopy(count_matrix)
        return C
    
    def get_ranks(self, A, B, words, word_weight_dict):
    #    i/p: A: matrix of simalarity scores between sentences
    #             row: sentences    column: sentences
    #             type: numpy array
    #         B: Normalized count matrix of words per sentences
    #             row: sentences    column: words
    #             type: numpy array
    #    o/p: P_t: List of calculated final ranks of sentences
    #             type: list
        no_sentences = A.shape[0]
        alpha = 0.6
        d=0.8
        P_tminus1 = [1]*no_sentences
        while True:
            P = self.eq_2(A,P_tminus1,d)
            Q = self.eq_3(B,P, words, word_weight_dict)
            P_star = self.eq_4(B,Q)
            P_t = self.eq_5(P,P_star,alpha)
            if self.get_diff(P, P_tminus1) < 10**-5:
                break
            P_tminus1 = copy.deepcopy(P_t)
        return P_t

# Post Processing

In [0]:
class summary_post_processing:
    def __init__(self):
        mf = misc_functions()
        return
    
    def get_indexes(self, sentence_ranks, no_summary_sentences):
    #    i/p: sentence_ranks = List of ranks of sentences.
    #             type: list
    #         no_summary_sentences: No. of sentences in reference summary.
    #             type: int
    #    o/p: sentence_index: Indexes of top-N ranked sentences. Where N =  no_summary_sentences
    #             type: list 
    
        sentence_index = []
        temp_list = copy.deepcopy(sentence_ranks)
        
        # No. of sentences in a document.
        no_sentences = len(sentence_ranks)
        if no_summary_sentences > no_sentences:
            no_summary_sentences = no_sentences
        for j in range(0, no_summary_sentences):
            index = temp_list.index(max(temp_list))
            sentence_index.append(index)
            temp_list[index] = 0
            
        # Arranging sentences indexes
        sentence_index.sort()
        
        return sentence_index
    
    def summary_generation(self, sentence_list, sentence_index):
    #    i/p: sentence_list = List of ranks of sentences.
    #             type: list
    #         sentence_index: Indexes of top-N ranked sentences. Where N =  no. of senteces in a
    #                         ref. summary
    #             type: list
    #    o/p: generated_summary: Model generated summary.
    #             type: string
    
        generated_summary = ""
      
        for index in sentence_index:
            generated_summary += sentence_list[index]
            generated_summary += " "
            
        return generated_summary
    
    def ref_summary(self, unprocessed_ref_summary):
    #    i/p: unprocessed_ref_summary = Reference summary which requires mionr processing.
    #             type: string
    #    o/p: reference_summary: Procesed reference summary.
    #             type: string
    
        reference_summary = copy.deepcopy(unprocessed_ref_summary)
        reference_summary.replace('.,','.')
        
        return reference_summary
    
    def graph_cluster_generation(self, graph_obj):
    #    i/p: graph_obj = Sentence-based graph of a sentence.
    #             type: GraphFrame object
    #    o/p: clusters: Map between clusters and sentence ids.
    #             type: Dictionary
    #             ex: {1: [2,8,3], 2: [1,7,9], 3: [4,5,6]}
    
      if graph_obj == None:
        return None
      
      clusters = {}
      cluster_df = graph_obj.connectedComponents()
      cluster_df_rows = cluster_df.collect()
      
      keys = []
      vertex_ids = []
      
      for row in cluster_df_rows:
        keys.append(int(row['component']))
        vertex_ids.append(int(row['id']))
      
      for key,vertex in zip(keys,vertex_ids):
        if key not in clusters:
          clusters[key] = [vertex]
          continue  
        clusters[key].append(vertex)
        
      return clusters
    
    def rearrange_cluster(self, cluster):
      arranged_cluster = {}
      new_cluster = []
      index = 0
    
      for cid in list(cluster.keys()):
      
        if len(cluster[cid]) < 2:
          new_cluster.append(cluster[cid][0])
          continue
        
        arranged_cluster[index] = copy.deepcopy(cluster[cid])
        index += 1
      
      if new_cluster != []:
        arranged_cluster[index] = new_cluster
        
      return arranged_cluster
    
    def post_sentence_rank(self, graph_obj, word_sentence_ranks = []):
      mf = misc_functions()
      cluster_sentence_ranks = []
      sentence_ranks = []
      vertices = []
      
      vertices_df = graph_obj.vertices.collect()
      for row in vertices_df:
        vertices.append(int(row['id']))
        
      vertices.sort()
      if word_sentence_ranks == []:
        print("I'm here!!!!")
        word_sentence_ranks = [0] * len(vertices)
        
      #Calculating Rank of vertex based on the cluster
      for vertex_id in vertices:
        cluster_sentence_ranks.append(mf.calculate_rank_cluster(vertex_id, graph_obj))
      
      #Combining clustering rank with the word-influced rank
      for index in range(len(vertices)):
        sentence_ranks.append(cluster_sentence_ranks[index] + word_sentence_ranks[index])
        
      return sentence_ranks
    
    def sentence_selection_cluster(self, cluster, no_summary_sentences, graph_obj, word_sentence_ranks = []):
      mf = misc_functions()
      sentence_index = []
      cluster_sentence_ranks = []
      
      no_sentence_doc = len(word_sentence_ranks)
      
      if no_summary_sentences > no_sentence_doc:
        no_summary_sentences = no_sentence_doc
        
      try:
        #Rearranging clusters
        cluster = self.rearrange_cluster(cluster)

        #Getting final rank of sentence by combinig cluster rank and word influnced rank
        sentence_ranks = self.post_sentence_rank(graph_obj, word_sentence_ranks)

        #Sorting nodes in lists of cluster
        for cid, sentence_list in zip(list(cluster.keys()),list(cluster.values())):
          cluster[cid] = mf.rankwise_sort(sentence_list, sentence_ranks)

        no_clusters = len(list(cluster.keys()))
        remaining_sentences = (no_summary_sentences)%(no_clusters)
        sentence_per_cluster = math.floor(no_summary_sentences/no_clusters)
        tempp = remaining_sentences
        temp_cluster = copy.deepcopy(cluster)

        for cid in list(temp_cluster.keys()):

          if sentence_per_cluster > len(temp_cluster[cid]):

            diff = sentence_per_cluster - len(temp_cluster[cid])
            remaining_sentences += diff
            sentence_index += temp_cluster[cid]
            temp_cluster[cid] = []
            continue

          for i in range(sentence_per_cluster):
            sentence_index.append(temp_cluster[cid].pop(0))

        reverse_index = -1
        while remaining_sentences != 0:
          index = list(temp_cluster.keys())[reverse_index]

          if temp_cluster[index] == []:
            reverse_index -= 1
            continue

          if remaining_sentences >= len(temp_cluster[index]):
            sentence_index += temp_cluster[index]
            remaining_sentences -= len(temp_cluster[index])
            temp_cluster[index] = []
            reverse_index -= 1
            continue

          if remaining_sentences < len(temp_cluster[index]):
            for i in range(remaining_sentences):
              sentence_index.append(temp_cluster[index].pop(0))
            remaining_sentences = 0
            reverse_index -= 1
            continue

      except AttributeError:
        sentence_index = self.get_indexes(word_sentence_ranks, no_summary_sentences)
          
      sentence_index.sort()
      
      return sentence_index

# Evaluation

In [0]:
class summary_evaluation:
    def __init__(self):
        return
    
    def rouge_measure(self, generated_summary, reference_summary, sub_cat = 'r'):
    #    i/p: sentence_ranks = List of ranks of sentences.
    #             type: list
    #         reference_summary: Procesed reference summary.
    #             type: string
    #         sub_cat: Sub-category for rouge.
    #                  'r' -> Recall, 'p' -> Precision, 'f' -> F1-score
    #             type: charcter
    #    o/p: rouge_scores_dict: Contains all three rouge scores: rouge-1, rouge-2, rouge-l
    #             type: dictionary
        rouge = Rouge()
        rouge_scores_dict = {}
        scores = rouge.get_scores(generated_summary, reference_summary, avg=True)
        rouge_scores_dict['rouge-1'] = scores['rouge-1'][sub_cat]
        rouge_scores_dict['rouge-2'] = scores['rouge-2'][sub_cat]
        rouge_scores_dict['rouge-l'] = scores['rouge-l'][sub_cat]
        return rouge_scores_dict
      
    def rouge_statistics(self,m_rouge_scores, need_list = False):
    #    i/p: rouge_scores_dict: Contains all three rouge scores: rouge-1, rouge-2, rouge-l
    #             type: dictionary
    #         need_list: Set true if want tje list of all  rouge scores
    #             type: dictionary
    #    o/p: rouge_stats_dict: Min, Max, Average of all three rouge scores: rouge-1, rouge-2, rouge-l for all documents
    #             type: dictionary
    #         rouge_list: Contains list of score lists for rouge-1, rouge-2, rouge-l
    #             type: list
    #    If need_list is true then o/p will be a list contains rouge_stats_dict and rouge_list
    
      rouge_stats_dict = {}
      rouge_1 = []
      rouge_2 = []
      rouge_l = []
      
      no_documents = len(m_rouge_scores)
      
      for dicti in m_rouge_scores:
        rouge_1.append(dicti["rouge-1"])
        rouge_2.append(dicti['rouge-2'])
        rouge_l.append(dicti["rouge-l"])
        
      rouge_stats_dict["rouge-1"] = round((sum(rouge_1)/no_documents),4)
      rouge_stats_dict["rouge-2"] = round((sum(rouge_2)/no_documents),4)
      rouge_stats_dict["rouge-l"] = round((sum(rouge_l)/no_documents),4)
      
      rouge_stats_dict["max_rouge_1"] = round(max(rouge_1),4)
      rouge_stats_dict["max_rouge_2"] = round(max(rouge_2),4)
      rouge_stats_dict["max_rouge_l"] = round(max(rouge_l),4)
      
      rouge_stats_dict["min_rouge_1"] = round(min(rouge_1),4)
      rouge_stats_dict["min_rouge_2"] = round(min(rouge_2),4)
      rouge_stats_dict["min_rouge_l"] = round(min(rouge_l),4)
      
      rouge_stats_dict["max_rouge_1_doc_id"] = round(rouge_1.index(max(rouge_1)),4)
      rouge_stats_dict["max_rouge_2_doc_id"] = round(rouge_2.index(max(rouge_2)),4)
      rouge_stats_dict["max_rouge_l_doc_id"] = round(rouge_l.index(max(rouge_l)),4)
      
      rouge_stats_dict["min_rouge_1_doc_id"] = round(rouge_1.index(min(rouge_1)),4)
      rouge_stats_dict["min_rouge_2_doc_id"] = round(rouge_2.index(min(rouge_2)),4)
      rouge_stats_dict["min_rouge_l_doc_id"] = round(rouge_l.index(min(rouge_l)),4)
      
      if need_list:
        return [rouge_stats_dict, [rouge_1, rouge_2, rouge_l]]
      
      return rouge_stats_dict

# Implementation

In [0]:
mf = misc_functions()
dp = data_preprocessing()
fe = feature_extraction()
rc = rank_calculation()
spp = summary_post_processing()
se = summary_evaluation()

In [0]:
sentence_table_name = 'wiki_how_all_sentences'
summary_table_name = 'wiki_how_all_summary'
m_doc_id = []
m_sentence_lemma_list = []
m_kw_sentence_lemma_list = []
m_nouns = []
m_words = []
m_count_matrix = []
m_keywords = []
m_word_weight_dict = []
m_similarity_matrix = []
mul_doc_ranks = []
m_sentence_index = []
m_rouge_scores = []
m_graph = []
m_tf_idf_score = []
rejected_sentences = []

In [0]:
m_generated_summary = []
m_reference_summary = []
df_summary = sqlContext.sql("SELECT * FROM "+summary_table_name)
summary_collection = df_summary.collect()
m_unprocessed_ref_summary = [str(row['summary']) for row in summary_collection]
no_summary_sentences = [int(row['sentence_count']) for row in summary_collection]

In [0]:
sentence_lists = mf.table_to_sentence_list(sentence_table_name)

In [0]:

# DATA PRE-PROCESSING
i = 0
for doc_sentence_list in sentence_lists:
#     print(i)
    
    early_porced_doc_sentence_list = dp.early_preprocessing(doc_sentence_list)
    sentence_tokens_list = dp.word_tokenization(early_porced_doc_sentence_list)
    sentence_lemma_list = dp.lemmatization(sentence_tokens_list)
    
    # sentence lemma list for keyword extraction
    m_kw_sentence_lemma_list.append(copy.deepcopy(sentence_lemma_list))
    
    sentence_lemma_list = dp.stop_word_removal(sentence_lemma_list)
    # sentence lemma list
    
    m_sentence_lemma_list.append(sentence_lemma_list)
    
    similarity_matrix = fe.sentenece_similarity_calculator(sentence_lemma_list)
    # similarity matrix
    m_similarity_matrix.append(similarity_matrix)

    # POS Tagging
    nouns, words = dp.pos_tagging(sentence_lemma_list)

    # noun list
    m_nouns.append(nouns)
    
    # word list
    m_words.append(words)

    i += 1 

In [0]:
# count matrix
for sentence_lemma_list in m_sentence_lemma_list:
  count_matrix = fe.word_freq_counter(sentence_lemma_list, words)
  m_count_matrix.append(count_matrix)
# KEYWORD GENERATION
for sentence_list in m_kw_sentence_lemma_list:
    # Finding keywords for each document
    keywords = fe.keyword_extraction('rake', sentence_list)
    m_keywords.append(keywords)
i = 0
for nn, kw, cm, wr in zip(m_nouns, m_keywords, m_count_matrix, m_words):
    # Calculating word weightage
    word_weight_dict = fe.word_weight_calculation(nn, kw, cm, wr)
    m_word_weight_dict.append(word_weight_dict)

In [0]:
#Graph Generation
for similarity_matrix in m_similarity_matrix:
  graph = rc.graph_generation(similarity_matrix)
  m_graph.append(graph)

In [0]:
# PROCESSING (RANK CALCULATION)
i = 0
for doc_id in range(0, len(sentence_lists)):
#     print(i)
#     i += 1 
    A = rc.get_A(m_similarity_matrix[doc_id])   #dimensions = number of sentences X number of sentences
    C = rc.get_C(m_count_matrix[doc_id])   #dimensions = number of sentences X number of words
    B = rc.get_B(C)   #dimensions = number of sentences X number of words
    P_t = rc.get_ranks(A, B, m_words[doc_id], m_word_weight_dict[doc_id])
    mul_doc_ranks.append(P_t)

### W/O Graph clstering

In [0]:
# POST-PROCESSING
i = 0
for index in range(0, len(sentence_lists)):
#     print(i)
    i += 1  
    sentence_ranks = mul_doc_ranks[index]
    sentence_list = sentence_lists[index]
    sentence_index = spp.get_indexes(sentence_ranks, no_summary_sentences[index])
    m_sentence_index.append(sentence_index)
    generated_summary = spp.summary_generation(sentence_list, sentence_index)
    m_generated_summary.append(generated_summary)
    unprocessed_ref_summary = m_unprocessed_ref_summary[index]
    reference_summary = spp.ref_summary(unprocessed_ref_summary)
    m_reference_summary.append(reference_summary)

In [0]:
# EVALUATION
for generated_summary,reference_summary in zip(m_generated_summary, m_reference_summary):
    rouge_scores_dict = se.rouge_measure(generated_summary, reference_summary, sub_cat = 'r')
    m_rouge_scores.append(rouge_scores_dict)

In [0]:
mf.print_rouge_statistics(se.rouge_statistics(m_rouge_scores)) # 10-50

------------------------------------------------------------------------------------
 | ROUGE-1 | Document | ROUGE-2 | Document | ROUGE-L | Document |
------------------------------------------------------------------------------------
 AVG | 0.4121 | NA | 0.1116 | NA | 0.3523 | NA |
------------------------------------------------------------------------------------
 MIN | 0.0000 | 88 | 0.0000 | 06 | 0.0000 | 88 |
------------------------------------------------------------------------------------
 MAX | 0.7778 | 93 | 0.5294 | 93 | 0.7692 | 93 |
------------------------------------------------------------------------------------

In [0]:
mf.print_rouge_statistics(se.rouge_statistics(m_rouge_scores)) # 50-100

------------------------------------------------------------------------------------
 | ROUGE-1 | Document | ROUGE-2 | Document | ROUGE-L | Document |
------------------------------------------------------------------------------------
 AVG | 0.4492 | NA | 0.1194 | NA | 0.3742 | NA |
------------------------------------------------------------------------------------
 MIN | 0.0000 | 24 | 0.0000 | 20 | 0.0000 | 24 |
------------------------------------------------------------------------------------
 MAX | 0.6591 | 57 | 0.3056 | 16 | 0.5714 | 63 |
------------------------------------------------------------------------------------

In [0]:
mf.print_rouge_statistics(se.rouge_statistics(m_rouge_scores)) # 100-150

------------------------------------------------------------------------------------
 | ROUGE-1 | Document | ROUGE-2 | Document | ROUGE-L | Document |
------------------------------------------------------------------------------------
 AVG | 0.4850 | NA | 0.1304 | NA | 0.378 | NA |
------------------------------------------------------------------------------------
 MIN | 0.2566 | 95 | 0.0179 | 95 | 0.2065 | 95 |
------------------------------------------------------------------------------------
 MAX | 0.6414 | 67 | 0.3158 | 37 | 0.5435 | 38 |
------------------------------------------------------------------------------------

In [0]:
mf.print_rouge_statistics(se.rouge_statistics(m_rouge_scores)) # 150-200

------------------------------------------------------------------------------------
 | ROUGE-1 | Document | ROUGE-2 | Document | ROUGE-L | Document |
------------------------------------------------------------------------------------
 AVG | 0.4761 | NA | 0.1171 | NA | 0.3654 | NA |
------------------------------------------------------------------------------------
 MIN | 0.2105 | 58 | 0.0000 | 58 | 0.1224 | 58 |
------------------------------------------------------------------------------------
 MAX | 0.6868 | 65 | 0.3259 | 62 | 0.5616 | 36 |
------------------------------------------------------------------------------------

In [0]:
mf.print_rouge_statistics(se.rouge_statistics(m_rouge_scores)) # 200

------------------------------------------------------------------------------------
 | ROUGE-1 | Document | ROUGE-2 | Document | ROUGE-L | Document |
------------------------------------------------------------------------------------
 AVG | 0.4487 | NA | 0.1004 | NA | 0.3536 | NA |
------------------------------------------------------------------------------------
 MIN | 0.0849 | 30 | 0.0000 | 20 | 0.0270 | 30 |
------------------------------------------------------------------------------------
 MAX | 0.7053 | 05 | 0.3016 | 05 | 0.5785 | 05 |
------------------------------------------------------------------------------------

### Graph Cluster based results

In [0]:
# Graph Cluster Generation
for graph in m_graph:
  m_cluster.append(spp.graph_cluster_generation(graph))

In [0]:
cl_m_sentence_index = []
cl_m_generated_summary = []
cl_m_rouge_scores = []

In [0]:
for i in range(100):
  val = []
  try:
    if type(m_cluster[i]) == None:
      print(str(i)+": NoneType")
      continue
    temp = list(m_cluster[i].values())
    for j in temp:
      val += j
    print(str(i)+": "+str(len(val)))
    if i % 10 == 9:
      print("=========================================")
      print(str(i+1) + " to " + str(i+10))
      print("=========================================")
  except KeyError: 
    print(str(i)+": Missing")

In [0]:
# POST-PROCESSING
i = 0
for index in range(0, len(sentence_lists)):
    start=datetime.now()
    print(i)
    i += 1  
    graph_obj = m_graph[index]
    cluster = m_cluster[index]
    cl_sentence_ranks = mul_doc_ranks[index]
    cl_sentence_list = sentence_lists[index]
    cl_sentence_index = spp.sentence_selection_cluster(cluster, no_summary_sentences[index], graph_obj, cl_sentence_ranks)
    cl_m_sentence_index.append(cl_sentence_index)
    cl_generated_summary = spp.summary_generation(cl_sentence_list, cl_sentence_index)
    cl_m_generated_summary.append(cl_generated_summary)
    cmin = time.localtime(time.time())[4]
    csec = time.localtime(time.time())[5]
    print("Process Time: "+str(datetime.now()-start))

In [0]:
# EVALUATION
for cl_generated_summary,reference_summary in zip(cl_m_generated_summary, m_reference_summary):
    cl_rouge_scores_dict = se.rouge_measure(cl_generated_summary, reference_summary, sub_cat = 'r')
    cl_m_rouge_scores.append(cl_rouge_scores_dict)

In [0]:
se.rouge_statistics(cl_m_rouge_scores)

In [0]:
mf.print_rouge_statistics(se.rouge_statistics(cl_m_rouge_scores)) #200

In [0]:
mf.print_rouge_statistics(se.rouge_statistics(cl_m_rouge_scores)) #150-200

------------------------------------------------------------------------------------
 | ROUGE-1 | Document | ROUGE-2 | Document | ROUGE-L | Document |
------------------------------------------------------------------------------------
 AVG | 0.4296 | NA | 0.0828 | NA | 0.3333 | NA |
------------------------------------------------------------------------------------
 MIN | 0.1587 | 76 | 0.0000 | 16 | 0.1020 | 58 |
------------------------------------------------------------------------------------
 MAX | 0.6103 | 62 | 0.2320 | 65 | 0.5205 | 36 |
------------------------------------------------------------------------------------

In [0]:
mf.print_rouge_statistics(se.rouge_statistics(cl_m_rouge_scores)) #100-150

------------------------------------------------------------------------------------
 | ROUGE-1 | Document | ROUGE-2 | Document | ROUGE-L | Document |
------------------------------------------------------------------------------------
 AVG | 0.4397 | NA | 0.0915 | NA | 0.3505 | NA |
------------------------------------------------------------------------------------
 MIN | 0.1351 | 68 | 0.0135 | 42 | 0.1250 | 68 |
------------------------------------------------------------------------------------
 MAX | 0.6167 | 76 | 0.2526 | 37 | 0.5714 | 13 |
------------------------------------------------------------------------------------

In [0]:
mf.print_rouge_statistics(se.rouge_statistics(cl_m_rouge_scores)) #50-100

------------------------------------------------------------------------------------
 | ROUGE-1 | Document | ROUGE-2 | Document | ROUGE-L | Document |
------------------------------------------------------------------------------------
 AVG | 0.4219 | NA | 0.0946 | NA | 0.3572 | NA |
------------------------------------------------------------------------------------
 MIN | 0.1000 | 23 | 0.0000 | 20 | 0.1111 | 23 |
------------------------------------------------------------------------------------
 MAX | 0.6591 | 57 | 0.3125 | 39 | 0.5231 | 98 |
------------------------------------------------------------------------------------

In [0]:
mf.print_rouge_statistics(se.rouge_statistics(cl_m_rouge_scores)) #10-50

------------------------------------------------------------------------------------
 | ROUGE-1 | Document | ROUGE-2 | Document | ROUGE-L | Document |
------------------------------------------------------------------------------------
 AVG | 0.3942 | NA | 0.0952 | NA | 0.3436 | NA |
------------------------------------------------------------------------------------
 MIN | 0.0833 | 88 | 0.0000 | 06 | 0.0909 | 88 |
------------------------------------------------------------------------------------
 MAX | 0.7222 | 93 | 0.4118 | 93 | 0.7692 | 93 |
------------------------------------------------------------------------------------